## Package Import

In [67]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression


## Data Import and Preprocessing

In [ ]:
full_path = 'C:\\Users\\RJEN0307\\Desktop\\Rasmus, Bachelorprojekt\\Data\\1_csv'

patient_numbers = []
patient_data = {}

for file in os.listdir(full_path):
    filename = os.fsdecode(file)
    patient_number = filename.split('_')[0] 
    patient_numbers.append(patient_number)
    patient_file_dir = os.path.join(full_path, filename)
    data = pd.read_csv(patient_file_dir)

    data.rename(columns={'Unnamed: 0': 'Index'}, inplace=True)
    data['Event'] = data['Event'].map({'R': 0, 'M': 1, 'F': 2})

    columns_to_remove = [col for col in data.columns if '_N' in col]
    data = data.drop(columns=columns_to_remove)

    resting_data = data[data['Event'] == 0]
    moving_data = data[data['Event'] == 1]
    familiar_data = data[data['Event'] == 2]
    target_count = min(moving_data.shape[0], familiar_data.shape[0])
    resting_data = resting_data.iloc[-target_count:]
    resting_data = resting_data.reset_index(drop=True)
    moving_data = moving_data.reset_index(drop=True)
    familiar_data = familiar_data.reset_index(drop=True)
    balanced_data = pd.concat([resting_data, moving_data, familiar_data]).reset_index(drop=True)

    standarize_list = ['PSD Delta', 'PSD Theta',  'PSD Alpha', 'PSD Beta', 'PSD Gamma', 'PSD SE', 'PSD MSF', 'PSD Sef90', 'PSD Sef95', 'PE', 'wSMI', 'Kolmogorov', 'Freq_Slope mean', 'Freq_Slope std']

    scaler = StandardScaler()
    balanced_data[standarize_list] = scaler.fit_transform(balanced_data[standarize_list])
    patient_data[patient_number] = balanced_data
    
    # show patient 10
    print(patient_data['p10'].head())

   Index  Event  PSD Delta  PSD Theta  PSD Alpha  PSD Beta  PSD Gamma  \
0   1500      0  -0.504771  -1.555079  -1.113242 -0.866272  -0.493307   
1   1501      0  -1.207551  -1.285578  -0.421632 -0.046604  -0.189003   
2   1502      0   2.094303  -0.218608  -0.972915 -1.282925  -0.468644   
3   1503      0   1.165314  -0.893731   0.395907 -1.449045  -1.025031   
4   1504      0  -0.710573  -0.297616  -1.010877 -0.883369  -1.267831   

     PSD SE   PSD MSF  PSD Sef90  PSD Sef95        PE      wSMI  Kolmogorov  \
0 -0.208458 -0.717660  -0.079449   0.452049 -0.135324 -0.296510    0.232978   
1  0.559548 -0.149165   0.738750   0.735636 -0.360464 -0.265355    0.580077   
2 -1.750320 -1.018849  -1.755760  -1.873936 -2.154975  0.022672   -0.688630   
3 -1.174411 -1.146854  -0.959331  -0.805472 -0.276526 -0.767685   -0.604848   
4  0.104931 -0.145400  -0.161088   0.230050 -0.436271 -0.012307    0.867331   

   Freq_Slope mean  Freq_Slope std  
0        -0.428410        0.710905  
1         1.

## Classification

### Threshold

In [ ]:
def classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, save_to_pdf=False, output_pdf_path='combined_patient_plots_final.pdf', final_plot=False):
    if save_to_pdf:
        pdf = PdfPages(output_pdf_path)

    combined_data = []
    combined_labels = []
    total_conf_matrix = np.zeros((2, 2))

    for patient in patients:
        data = patient_data[patient].copy()
        labels = data['Event'].apply(lambda x: 1 if x in [1, 2] else 0).values
        data_for_pca = data.drop(columns=['Event']).values

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_accuracies = []
        for train_index, test_index in kf.split(data_for_pca):
            X_train, X_test = data_for_pca[train_index], data_for_pca[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            
            pca = PCA(n_components=n_components_pca)
            X_train_pca = pca.fit_transform(X_train)
            X_test_pca = pca.transform(X_test)

            threshold = (X_train_pca[y_train == 0].mean() + X_train_pca[y_train == 1].mean()) / 2
            y_pred = (X_test_pca[:, 0] < threshold).astype(int)

            fold_accuracy = accuracy_score(y_test, y_pred)
            fold_accuracies.append(fold_accuracy)

            fold_conf_matrix = confusion_matrix(y_test, y_pred)
            total_conf_matrix += fold_conf_matrix

        if final_plot:
            X_pca_full = pca.fit_transform(data_for_pca)
            combined_data.append(X_pca_full)
            combined_labels.append(labels)

    if final_plot:
        combined_data = np.vstack(combined_data)
        combined_labels = np.concatenate(combined_labels)

        avg_conf_matrix = total_conf_matrix / len(patients)

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.scatter(combined_data[combined_labels == 0, 0], np.zeros_like(combined_data[combined_labels == 0, 0]), color='blue', label='Resting')
        ax.scatter(combined_data[combined_labels == 1, 0], np.zeros_like(combined_data[combined_labels == 1, 0]), color='red', label='Familiar/Medical')

        combined_threshold = (combined_data[combined_labels == 0, 0].mean() + combined_data[combined_labels == 1, 0].mean()) / 2
        ax.axvline(x=combined_threshold, color='green', linestyle='--', label='Combined Threshold')

        threshold_text = f"Threshold: {combined_threshold:.2f}"
        ax.text(0.05, 0.95, threshold_text, transform=ax.transAxes, fontsize=10, verticalalignment='top', color='black')

        ax.set_title("Separation of Resting and Familiar/Medical States for All Patients")
        ax.set_xlabel('PCA Component 1')
        ax.legend()
        ax.set_yticks([])

        conf_matrix_text = f"Average Confusion Matrix Over All Patients:\n"
        conf_matrix_text += f"TN: {avg_conf_matrix[0, 0]:.2f}, FP: {avg_conf_matrix[0, 1]:.2f}\n"
        conf_matrix_text += f"FN: {avg_conf_matrix[1, 0]:.2f}, TP: {avg_conf_matrix[1, 1]:.2f}"
        ax.text(0.95, 0.05, conf_matrix_text, transform=ax.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right')

        if save_to_pdf:
            pdf.savefig(fig)
            plt.close(fig)
        else:
            plt.show()

    if save_to_pdf:
        pdf.close()

# Example usage for all patients
patients = patient_data.keys()
classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, final_plot=True, save_to_pdf=True, output_pdf_path='PDF files\\Threshold.pdf')


### Logistic Regression

In [ ]:
def classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, save_to_pdf=False, output_pdf_path='combined_patient_plots_final.pdf', final_plot=False):
    if save_to_pdf:
        pdf = PdfPages(output_pdf_path)


    combined_data = []
    combined_labels = []
    total_conf_matrix = np.zeros((2, 2)) 


    for patient in patients:
        data = patient_data[patient].copy()
        labels = data['Event'].apply(lambda x: 1 if x in [1, 2] else 0).values  # Familiar/Medical as 1, Resting as 0
        data_for_pca = data.drop(columns=['Event']).values


        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_accuracies = []
        for train_index, test_index in kf.split(data_for_pca):
            X_train, X_test = data_for_pca[train_index], data_for_pca[test_index]
            y_train, y_test = labels[train_index], labels[test_index]


            pca = PCA(n_components=n_components_pca)
            X_train_pca = pca.fit_transform(X_train)
            X_test_pca = pca.transform(X_test)


            model = LogisticRegression()
            model.fit(X_train_pca, y_train)
            y_pred = model.predict(X_test_pca)


            fold_accuracy = accuracy_score(y_test, y_pred)
            fold_accuracies.append(fold_accuracy)


            fold_conf_matrix = confusion_matrix(y_test, y_pred)
            total_conf_matrix += fold_conf_matrix  


        if final_plot:
            X_pca_full = pca.fit_transform(data_for_pca)
            combined_data.append(X_pca_full)
            combined_labels.append(labels)


    if final_plot:
        combined_data = np.vstack(combined_data)
        combined_labels = np.concatenate(combined_labels)
        

        avg_conf_matrix = total_conf_matrix / len(patients)
        

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.scatter(combined_data[combined_labels == 0, 0], np.zeros_like(combined_data[combined_labels == 0, 0]), color='blue', label='Resting')
        ax.scatter(combined_data[combined_labels == 1, 0], np.zeros_like(combined_data[combined_labels == 1, 0]), color='red', label='Familiar/Medical')
        

        combined_model = LogisticRegression()
        combined_model.fit(combined_data[:, 0].reshape(-1, 1), combined_labels)
        combined_coef, combined_intercept = combined_model.coef_[0][0], combined_model.intercept_[0]
        combined_threshold = -combined_intercept / combined_coef
        ax.axvline(x=combined_threshold, color='green', linestyle='--', label='Combined Decision Boundary')
        

        ax.set_title("Combined Separation of Resting and Familiar/Medical States for All Patients")
        ax.set_xlabel('PCA Component 1')
        ax.legend()
        

        conf_matrix_text = f"Average Confusion Matrix Over Folds:\n"
        conf_matrix_text += f"TN: {avg_conf_matrix[0, 0]:.2f}, FP: {avg_conf_matrix[0, 1]:.2f}\n"
        conf_matrix_text += f"FN: {avg_conf_matrix[1, 0]:.2f}, TP: {avg_conf_matrix[1, 1]:.2f}"
        ax.text(0.95, 0.05, conf_matrix_text, transform=ax.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right')

        if save_to_pdf:
            pdf.savefig(fig)
            plt.close(fig)
        else:
            plt.show()

    if save_to_pdf:
        pdf.close()

# Example usage for all patients
patients = patient_data.keys()
classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, final_plot=True, save_to_pdf=True, output_pdf_path='PDF files\\combined_patient_plots_final.pdf')


### Logistic Regression & ROC-Curve

In [ ]:
def classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, save_to_pdf=False, output_pdf_path='combined_patient_plots_final.pdf', final_plot=False, individual_plots=False):
    if save_to_pdf:
        pdf = PdfPages(output_pdf_path)
    
    combined_data = []
    combined_labels = []
    total_conf_matrix = np.zeros((2, 2))

    all_true_labels = []
    all_scores = []

    for patient in patients:
        data = patient_data[patient].copy()

        labels = data['Event'].apply(lambda x: 1 if x in [1, 2] else 0).values 
        data_for_pca = data.drop(columns=['Event']).values

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_accuracies = []

        for train_index, test_index in kf.split(data_for_pca):
            X_train, X_test = data_for_pca[train_index], data_for_pca[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            
            pca = PCA(n_components=n_components_pca)
            X_train_pca = pca.fit_transform(X_train)
            X_test_pca = pca.transform(X_test)

            model = LogisticRegression()
            model.fit(X_train_pca, y_train)
            y_pred = model.predict(X_test_pca)

            y_scores = model.predict_proba(X_test_pca)[:, 1]

            all_true_labels.extend(y_test)
            all_scores.extend(y_scores)

            fold_accuracy = accuracy_score(y_test, y_pred)
            fold_accuracies.append(fold_accuracy)

            fold_conf_matrix = confusion_matrix(y_test, y_pred)
            total_conf_matrix += fold_conf_matrix

        average_accuracy = np.mean(fold_accuracies)

        if individual_plots:
            fig, ax = plt.subplots()
            ax.scatter(X_test_pca[y_test == 0, 0], np.zeros_like(X_test_pca[y_test == 0, 0]), color='blue', label='Resting')
            ax.scatter(X_test_pca[y_test == 1, 0], np.zeros_like(X_test_pca[y_test == 1, 0]), color='red', label='Familiar/Medical')
            ax.set_title(f'Patient {patient} PCA Component Separation')
            ax.set_xlabel('PCA Component 1')
            ax.legend()
            if save_to_pdf:
                pdf.savefig(fig)
                plt.close(fig)
            else:
                plt.show()

        if final_plot:
            X_pca_full = pca.fit_transform(data_for_pca)
            combined_data.append(X_pca_full)
            combined_labels.append(labels)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    fpr, tpr, thresholds = roc_curve(all_true_labels, all_scores)
    roc_auc = auc(fpr, tpr)

    ax1.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('Receiver Operating Characteristic (ROC) Curve')
    ax1.legend(loc='lower right')

    if final_plot:
        combined_data = np.vstack(combined_data)
        combined_labels = np.concatenate(combined_labels)
        
        avg_conf_matrix = total_conf_matrix / len(patients)
        
        ax2.scatter(combined_data[combined_labels == 0, 0], np.zeros_like(combined_data[combined_labels == 0, 0]), color='blue', label='Resting')
        ax2.scatter(combined_data[combined_labels == 1, 0], np.zeros_like(combined_data[combined_labels == 1, 0]), color='red', label='Familiar/Medical')
        
        model = LogisticRegression()
        model.fit(combined_data, combined_labels)
        
        x_min, x_max = combined_data[:, 0].min() - 1, combined_data[:, 0].max() + 1
        xx = np.linspace(x_min, x_max, 500).reshape(-1, 1)
        
        probs = model.predict_proba(xx)[:, 1]
        boundary_idx = np.abs(probs - 0.5).argmin()  
        decision_boundary = xx[boundary_idx]
        
        ax2.axvline(x=decision_boundary, color='green', linestyle='--', label='Decision Boundary')
        
        conf_matrix_text = f"Average Confusion Matrix Over Folds:\n"
        conf_matrix_text += f"TN: {avg_conf_matrix[0, 0]:.2f}, FP: {avg_conf_matrix[0, 1]:.2f}\n"
        conf_matrix_text += f"FN: {avg_conf_matrix[1, 0]:.2f}, TP: {avg_conf_matrix[1, 1]:.2f}"
        ax2.text(0.95, 0.05, conf_matrix_text, transform=ax2.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right')

        # Display logistic regression coefficient
        coef_text = f"Logistic Regression Coefficient: {model.coef_[0][0]:.2f}"
        ax2.text(0.05, 0.95, coef_text, transform=ax2.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='left')

        ax2.set_title("Combined Separation of Resting and Familiar/Medical States with Decision Boundary")
        ax2.set_xlabel('PCA Component 1')
        ax2.legend()

    if save_to_pdf:
        pdf.savefig(fig)
        plt.close(fig)
    else:
        plt.show()

    if save_to_pdf:
        pdf.close()

# Example usage for all patients
patients = patient_data.keys()
classification_with_cross_validation(patient_data, patients, n_components_pca=1, n_splits=2, final_plot=True, individual_plots=False, save_to_pdf=True, output_pdf_path='PDF files\\LogsticRocPlot.pdf')
